In [2]:
#!pip install --upgrade keras-cv tensorflow
#!pip install --upgrade keras
!pip install keras_tuner
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import keras
import datetime
%load_ext tensorboard
import keras_tuner as kt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.4 MB/s eta 0:00:00
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

"from google.colab import drive\ndrive.mount('/content/gdrive')"

In [4]:
cifar10 = tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = cifar10


170498071/170498071 [==============================] - 13s 0us/step


In [5]:
X_train_full = tf.image.rgb_to_grayscale(X_train_full)
X_test = tf.image.rgb_to_grayscale(X_test)

In [6]:
X_train_full.shape

TensorShape([50000, 32, 32, 1])

In [7]:
"""to_grayscale = keras.layers.preprocessing.Grayscale()
X_train_full = to_grayscale(X_train_full)
X_test = to_grayscale(X_test)"""

'to_grayscale = keras.layers.preprocessing.Grayscale()\nX_train_full = to_grayscale(X_train_full)\nX_test = to_grayscale(X_test)'

In [8]:
X_train, y_train = X_train_full[:-10000], y_train_full[:-10000]
X_valid, y_valid = X_train_full[-10000:], y_train_full[-10000:]

In [9]:
print(X_train[1][0][1])

tf.Tensor([133], shape=(1,), dtype=uint8)


In [10]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
class_names[int(y_train[5])]


<ipython-input-10-f94cb98b12e9>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  class_names[int(y_train[5])]


'automobile'

In [11]:



model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = [32, 32, 1]),
    tf.keras.layers.Dense(800, kernel_initializer = 'random_normal',bias_initializer = 'zeros' ,activation = "relu"),
    tf.keras.layers.Dense(700, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(600, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(400, kernel_initializer = 'random_normal',bias_initializer = 'zeros', activation = "relu"),
    tf.keras.layers.Dense(10, activation = "softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 800)               820000    
                                                                 
 dense_1 (Dense)             (None, 700)               560700    
                                                                 
 dense_2 (Dense)             (None, 600)               420600    
                                                                 
 dense_3 (Dense)             (None, 600)               360600    
                                                                 
 dense_4 (Dense)             (None, 600)               360600    
                                                                 
 dense_5 (Dense)             (None, 400)               2

In [12]:
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = "accuracy")

In [14]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoints = tf.keras.callbacks.ModelCheckpoint('./checkpoint.model.keras')
history = model.fit(X_train, y_train, epochs = 30, validation_data = (X_valid, y_valid), callbacks=[tensorboard_callback, checkpoints])

Epoch 1/30
1250/1250 [==============================] - 8s 6ms/step - loss: nan - accuracy: 0.0996 - val_loss: nan - val_accuracy: 0.1014
Epoch 2/30
1250/1250 [==============================] - 6s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 3/30
1250/1250 [==============================] - 7s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 4/30
1250/1250 [==============================] - 6s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 5/30
1250/1250 [==============================] - 7s 6ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 6/30
1250/1250 [==============================] - 6s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 7/30
1250/1250 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.0997 - val_loss: nan - val_accuracy: 0.1014
Epoch 8/30
1250/1250 [============

In [15]:
weights = model.get_layer("dense_1").get_weights()

In [16]:
weights[1].shape

(700,)

In [17]:
# Load the TensorBoard notebook extension


In [18]:
def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value = 0, max_value = 10, default = 2)
  n_neurons = hp.Int("n_neurons", min_value = 10, max_value = 500, default = 3)
  learning_rate = hp.Float("learning_rate", min_value = 1e-4, max_value = 1e-2, sampling = "log")
  optimizer = hp.Choice("optimizer", values = ["sgd", "adam"])
  if optimizer == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
  else:
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape = [32,32,1]))
  for i in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation = "relu"))
  model.add(tf.keras.layers.Dense(10, activation = "softmax"))
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
  return model



In [19]:
random_search_tuner = kt.RandomSearch(build_model,
                                  objective = "accuracy",
                                  max_trials = 100,
                                  seed = 42,
                                  directory = "cifarproject",
                                  project_name = "random_search_runs")
random_search_tuner.search(X_train, y_train, epochs = 5, validation_data = (X_valid, y_valid))

Trial 100 Complete [00h 00m 42s]
accuracy: 0.147025004029274

Best accuracy So Far: 0.36492499709129333
Total elapsed time: 00h 58m 45s


In [ ]:
!rm -rf cifarproject/